In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv("TripAdvisor_RestauarantRecommendation.csv")

In [2]:
data

,Name,Street Address,Location,Type,Reviews,No of Reviews,Comments,Contact Number,Trip_advisor Url,Menu,Price_Range
0,Betty Lou's Seafood and Grill,318 Columbus Ave,"San Francisco, CA 94133-3908","Seafood, Vegetarian Friendly, Vegan Options",4.5 of 5 bubbles,243 reviews,NaN,+1 415-757-0569,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1,Coach House Diner,55 State Rt 4,"Hackensack, NJ 07601-6337","Diner, American, Vegetarian Friendly",4 of 5 bubbles,84 reviews,"Both times we were there very late, after 11 P...",+1 201-488-4999,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2,Table Talk Diner,2521 South Rd Ste C,"Poughkeepsie, NY 12601-5476","American, Diner, Vegetarian Friendly",4 of 5 bubbles,256 reviews,Waitress was very friendly but a little pricey...,+1 845-849-2839,https://www.tripadvisor.com//Restaurant_Review...,http://tabletalkdiner.com/menu/breakfast/,$$ - $$$
3,Sixty Vines,3701 Dallas Pkwy,"Plano, TX 75093-7777","American, Wine Bar, Vegetarian Friendly",4.5 of 5 bubbles,235 reviews,Not sure why I went there for the second time....,+1 469-620-8463,https://www.tripadvisor.com//Restaurant_Review...,https://sixtyvines.com/menu/plano-tx/,$$ - $$$
4,The Clam Bar,3914 Brewerton Rd,"Syracuse, NY 13212","American, Bar, Seafood",4 of 5 bubbles,285 reviews,Doesn't look like much from the outside but wa...,+1 315-458-1662,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
...,...,...,...,...,...,...,...,...,...,...,...
3057,Indigo Kitchen & Ale House,2902 164th St SW,"Lynnwood, WA 98087-3201","American, Bar, Vegetarian Friendly",4.5 of 5 bubbles,198 reviews,We had to wait a few minutes to get it but it ...,+1 425-741-8770,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
3058,City Vineyard,233 West Street,"New York City, NY 10013","American, Bar, Seafood",4.5 of 5 bubbles,374 reviews,"Came here to see Andrea Gibson perform, we wer...",+1 646-677-8350,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
3059,BRIO Tuscan Grille,3710 US Highway 9,"Freehold, NJ 07728-4801","Steakhouse, Italian, Bar",4 of 5 bubbles,211 reviews,We come to Brio once a month. They are good fo...,+1 732-683-1045,https://www.tripadvisor.com//Restaurant_Review...,http://www.brioitalian.com/menus.html,$$ - $$$
3060,Maywood Pancake house,92 W Pleasant Ave,"Maywood, NJ 07607-1336","American, Vegan Options, Gluten Free Options",4 of 5 bubbles,87 reviews,Stopped in yesterday for Breakfast. When in Je...,+1 201-880-7842,https://www.tripadvisor.com//Restaurant_Review...,http://www.maywoodpancakehouse.com/menu.html,$$ - $$$


In [3]:
data.columns

Index(['Name', 'Street Address', 'Location', 'Type', 'Reviews',
       'No of Reviews', 'Comments', 'Contact Number', 'Trip_advisor Url',
       'Menu', 'Price_Range'],
      dtype='object')

In [4]:
data2 = data[["Name","Comments"]]

In [5]:
data2.isna().sum()

Name          0
Comments    615
dtype: int64

In [6]:
data2["Comments"] = data["Comments"].fillna("")

/tmp/ipykernel_20641/2261508245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2["Comments"] = data["Comments"].fillna("")


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english",max_features=10000 )
tfidf_matrix= tfidf.fit_transform(data2["Comments"])
tfidf_matrix.shape

(3062, 6245)

In [9]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(data2.index,index=data2['Name']).drop_duplicates()
indices

Name
Betty Lou's Seafood and Grill       0
Coach House Diner                   1
Table Talk Diner                    2
Sixty Vines                         3
The Clam Bar                        4
                                 ... 
Indigo Kitchen & Ale House       3057
City Vineyard                    3058
BRIO Tuscan Grille               3059
Maywood Pancake house            3060
Porto Leggero                    3061
Length: 3062, dtype: int64

In [10]:
def res_recommender(Name,cosine_sin=cosine_sim,data=data2,indices=indices):
    idx = indices[Name]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score, key=lambda x : x[1], reverse =True)
    sim_score = sim_score[1:11]
    restuarent_indices = [i[0] for i in sim_score]
    return data["Name"].iloc[restuarent_indices]
res_recommender("The Clam Bar")

1075    Toulouse Cafe and Bar (Dallas)
1842                          Lapellah
1156               LongHorn Steakhouse
2753        Copper Top Tavern Camillus
2018               Terra Mediterranean
2194                      P.F. Chang's
741                Shanghai You Garden
744           Gringo's Mexican Kitchen
2416                 Logan's Roadhouse
1648                        Applebee's
Name: Name, dtype: object